#  Cointab Supply Chain Management Challenge

##  Business Scenario
You are a data analyst working for a large ecommerce company in India (referred to as **Company X**). The company receives a few thousand orders daily and aims to deliver them quickly. To manage deliveries, they’ve partnered with multiple courier companies that charge per shipment.

Delivery charges depend on:
- **Weight of the product**
- **Distance** between the warehouse (pickup location) and the customer’s delivery address (destination)

>  On average, the delivery charge is **₹100** per shipment. With ~1,00,000 shipments monthly, the cost becomes ₹1 crore.

To optimize costs, **Company X wants to verify** whether the courier company’s charges per order are accurate.


##  Input Data

### 🔹 LHS – Company X’s Internal Data
1. **Website Order Report**: Contains `Order ID` and products (SKUs) in each order. (Order ID is the key.)
2. **SKU Master**: Includes gross weight of each SKU.
3. **Warehouse Pincode to Customer Pincode Mapping**: Helps identify the **Delivery Zone (A/B/C/D/E)**.

#### Weight Slab Examples:
- 400 grams → 0.5 KG
- 950 grams → 1.0 KG
- 1.0 KG → 1.0 KG
- 2.2 KG → 2.5 KG

### 🔹 RHS – Courier Company Invoice
- `AWB Number` (Courier ID)
- `Order ID`
- `Charged Weight`
- `Warehouse Pincode`
- `Customer Pincode`
- `Zone of Delivery`
- `Shipment Type`
- `Charges Billed`

###  Rate Card
Courier rate card based on:
- **Zone-wise charges**
- **Weight slabs (0.5 KG intervals)**
- Types: `Forward charges` and `RTO charges`

#### Charging Rule:
```
Total Charges = Fixed Charge (for first 0.5 KG)
              + Additional Charges (for each extra 0.5 KG)
```



##  Output Data
Create a CSV/Excel file with the following:

| Column                                  |
|-----------------------------------------|
| Order ID                                |
| AWB Number                              |
| Total Weight as per X (KG)              |
| Weight Slab as per X (KG)               |
| Total Weight as per Courier Company     |
| Weight Slab charged by Courier Company  |
| Delivery Zone as per X                  |
| Delivery Zone charged by Courier Company |
| Expected Charge as per X (Rs.)          |
| Charges Billed by Courier Company (Rs.) |
| Difference Between Expected and Billed  |

###  Summary Metrics
- **Correctly Charged Orders**
- **Overcharged Orders**
- **Undercharged Orders**
- **Total Invoice Amount**
- **Total Overcharge / Undercharge (Rs.)**


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# Load all provided Excel files
order_df = pd.read_excel("Company X - Order Report.xlsx")
sku_df = pd.read_excel("Company X - SKU Master.xlsx")
invoice_df = pd.read_excel("Courier Company - Invoice.xlsx")
pincode_zone_df = pd.read_excel("Company X - Pincode Zones.xlsx")
rate_df = pd.read_excel("Courier Company - Rates.xlsx")

In [4]:
# Merge order with SKU weights
order_df = order_df.merge(sku_df, on="SKU", how="left")
order_df["Total Weight (g)"] = order_df["Order Qty"] * order_df["Weight (g)"]

# Sum total weight per Order ID
order_weight = order_df.groupby("ExternOrderNo")["Total Weight (g)"].sum().reset_index()
order_weight.columns = ["Order ID", "Total Weight (g)"]
order_weight["Total Weight as per X (KG)"] = order_weight["Total Weight (g)"] / 1000

# Round up to nearest 0.5 kg slab
order_weight["Weight Slab as per X (KG)"] = np.ceil(order_weight["Total Weight as per X (KG)"] * 2) / 2

In [5]:
invoice_df.rename(columns={"Charged Weight": "Weight Slab charged by Courier Company (KG)"}, inplace=True)
full_df = invoice_df.merge(order_weight, on="Order ID", how="left")

# Map zone info
zone_map = pincode_zone_df.drop_duplicates(subset=["Warehouse Pincode", "Customer Pincode"])
zone_map.columns = ["Warehouse Pincode", "Customer Pincode", "Zone as per X"]
full_df = full_df.merge(zone_map, on=["Warehouse Pincode", "Customer Pincode"], how="left")

In [6]:
def calculate_expected_charge(row):
    weight = row['Weight Slab as per X (KG)']
    shipment_type = row['Type of Shipment']
    zone = row['Zone as per X']
    if pd.isnull(weight) or pd.isnull(zone): return np.nan
    zone = zone.lower()
    fixed_cost = 0
    additional_cost = 0

    if 'Forward' in shipment_type:
        fixed_cost += rate_df[f"fwd_{zone}_fixed"][0]
        additional_cost += rate_df[f"fwd_{zone}_additional"][0] * max(0, (weight - 0.5) / 0.5)

    if 'rto' in shipment_type.lower():
        fixed_cost += rate_df[f"rto_{zone}_fixed"][0]
        additional_cost += rate_df[f"rto_{zone}_additional"][0] * max(0, (weight - 0.5) / 0.5)

    return round(fixed_cost + additional_cost, 2)

In [7]:
full_df["Expected Charge as per X (Rs.)"] = full_df.apply(calculate_expected_charge, axis=1)
full_df["Difference Between Expected Charges and Billed Charges (Rs.)"] = full_df["Expected Charge as per X (Rs.)"] - full_df["Billing Amount (Rs.)"]

full_df["Charge Status"] = full_df["Difference Between Expected Charges and Billed Charges (Rs.)"].apply(
    lambda x: "Correctly Charged" if abs(x) < 1 else ("Overcharged" if x < 0 else "Undercharged")
)

In [8]:
summary = full_df.groupby("Charge Status").agg(
    Order_Count=("Order ID", "count"),
    Total_Discrepancy=("Difference Between Expected Charges and Billed Charges (Rs.)", "sum")
).reset_index()

# Save results
full_df.to_excel("order_level_analysis.xlsx", index=False)
summary.to_excel("summary_table.xlsx", index=False)

summary

,Charge Status,Order_Count,Total_Discrepancy
0,Correctly Charged,22,0.0
1,Overcharged,79,-4426.6
2,Undercharged,23,630.6


### Recommendations
- Automate this audit monthly to flag discrepancies and overcharges.
- Investigate courier partners and renegotiate rates for high-cost zones.
- Analyze high RTO zones and take steps to reduce reverse logistics costs.
- Use this model for pre-invoice verification at scale.
